# 基于生成器的协程(快被淘汰)

按照 Python文档的描述，所谓的「基于生成器的协程」指的是用 `yield from创` 创建建的生成器，并且还要搭配 `asyncio.coroutine 装饰器来使用。

讲的原因是新的 async 方法和原来的 yield 方法的接口基本一模一样

In [ ]:
def generator_func():
    yield 1

gen = generator_func()

print(gen)
print(sorted(set(dir(gen)) - set(dir(object))))

<generator object generator_func at 0x000001FC86A24BF0>
['__del__', '__iter__', '__name__', '__next__', '__qualname__', 'close', 'gi_code', 'gi_frame', 'gi_running', 'gi_suspended', 'gi_yieldfrom', 'send', 'throw']


In [ ]:
async def coroutine_func():
    await coroutine_func()

coro = coroutine_func()

print(coro)
print(sorted(set(dir(coro)) - set(dir(object))))

<coroutine object coroutine_func at 0x000001FC8640AF80>
['__await__', '__del__', '__name__', '__qualname__', 'close', 'cr_await', 'cr_code', 'cr_frame', 'cr_origin', 'cr_running', 'cr_suspended', 'send', 'throw']


![](./协程发展时间.png)

# `yield` 是一个表达式

In [ ]:
def show_yield_value():
    x = yield
    print(f"x is {x}")

g = show_yield_value()
next(g)

In [ ]:
next(g)

x is None


StopIteration: 

为生成器增加一个 `send()` 方法，该方法可以接受一个入参。

`send()` 方法顾名思义，将该参数发送给生成器，使生成器恢复运行的同时，将该入参作为 `yield` 表达式的值。

In [ ]:
def show_yield_value():
    print("start")
    x = yield
    print(f"x is {x}")

g = show_yield_value()
g.send(None) # 第一次只能传递 None 值

start


In [ ]:
g.send("hello")

x is hello


StopIteration: 

关于prime

对于刚创建好的生成器，总是需要在第一次的时候 `send(None)` 值，使其运行到 `yield` 的地方暂停，这个步骤术语称为prime。

这里prime做动词解的意思：PREPARESOMEBODYtopreparesomeoneford situationsothattheyknowwhattodo←一使准备好【应付某个情况】

prime在《流畅的Python》中文版被翻译为「预激」，感觉有点过于专业口其实这个步骤后续并没有那么重要（后面会解释），所以我就直接称为/激活，大家明白是什么意思就好

# `yield` 表达式的优先级

In [ ]:
def add_yield_value():
    # x = yield + 1 # 这种写法有 bug， 因为 yield 的原有的语法是 yield 后面的值，因此这一行等同于 yield 1
    # 这样写才行
    x = (yield) + 1
    print(f"x is {x}")

In [ ]:
g = add_yield_value()
g.send(None)

In [ ]:
g.send(1)

x is 2


StopIteration: 

# `send()` 用法总结

- `send()` 是生成器对象的方法
- 对于生成器对象 `g`，`next(g)` 等价于 `g.send(None)`
- 只有当生成器处在「暂停」状态时，才能传入非 `None` 的值
- `send()` 方法是为了协程而增加的APl，所以：
    - 如果将生成器视作协程，就应该只用 `send` 方法
    - 如果视作迭代器，就仍用 `next`

所以，后面我们统一都使用 `g.send(None)` 的方式，而不再采用 next(g) 的方式

In [ ]:
# 一个极简的echo
def gen_echo():
    while True:
        print((yield))

echo = gen_echo()
echo.send(None)

In [ ]:
for i in "一键三连":
    echo.send(i)

一
键
三
连


In [ ]:
# 直接传入 StopIteration 异常没有用,无法停止生成器
echo.send(StopIteration("stop"))

stop


# 使用 `close()` 结束生成器

当生成器作为迭代器来用的时候，它的生命周期取决于有多少元素可以迭代

而当作协程来用的时候，通常可以视作是在执行一个任务，我们希望任务的终止能够变得可控

新增的 `close()` 方法就是用来结束一个协程

In [ ]:
echo.close()
echo.send(1)

StopIteration: 

由于echo协程的内容非常简单，所以可以直接结束。

如果协程的代码比较复杂，它可能需要在结束的时候做一些善后处理，比如释放资源等。

类似于StopIteration的实现机制，结束协程也是靠异常来实现的：

In [ ]:
def gen_echo_v2():
    while True:
        try:
            x = yield
        except GeneratorExit:
            print("Exit, bye")
            return # `GeneratorExit` 异常要求强制退出
        else:
            print(x)

echo_v2 = gen_echo_v2()
echo_v2.send(None)

In [ ]:
for i in "一键三连":
    echo_v2.send(i)

一
键
三
连


In [ ]:
echo_v2.close()

Exit, bye


# 使用 `throw` 将异常抛给 `yield`

In [ ]:
def gen_echo_v3():
    while True:
        try:
            x = yield
        except GeneratorExit:
            print("Exit, bye")
            return # `GeneratorExit` 异常要求强制退出
        except KeyboardInterrupt:
            print("Ctrl+C, bye")
            # 注意这里没有 return，因为只有 `GeneratorExit` 异常要求强制退出
        else:
            print(x)

echo_v3 = gen_echo_v3()
echo_v3.send(None)

In [ ]:
for i in "一键三连":
    echo_v3.send(i)

一
键
三
连


In [ ]:
echo_v3.throw(KeyboardInterrupt)

Ctrl+C, bye


In [ ]:
for i in "一键三连":
    echo_v3.send(i)

一
键
三
连


In [ ]:
# 如果传递了协程没法处理的异常，会向上抛出，导致协程退出

echo_v3.throw(RuntimeError('error'))

RuntimeError: error

In [ ]:
echo_v3.send(1)

StopIteration: 

# 总结协程的几个功能点

In [ ]:
def coro_averager():
    """计算移动平均值"""
    count = 0
    total = 0
    avg = None
    while True:
        try:
            # yield 将 avg 返回回去，同时将 send() 的值保存到 value 中
            value = yield avg
        except GeneratorExit:
            print("Exit, bye")
            return total, count, avg
        else:
            total += value
            count += 1
            avg = total / count

1. 在 `yield` 的位置产出数据
2. 在 `yield` 的位置暂停
3. 在 `yield` 的位置恢复，并接受新的参数
4. 在 `yield` 的位置传入结束信号
5. 在 `yield` 的位置传入其它异常

In [ ]:
aver = coro_averager()
aver.send(None)

for i in range(1, 10):
    print(aver.send(i))

1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5
5.0


In [ ]:
print(aver.close())

Exit, bye
None
